### 輸入資料

In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow import keras
# from google.colab import drive

# drive.mount('/content/drive')
print("hello world")

hello world


In [2]:
import re
mypath = 'K:\\DataMiningProject\\Training_set\\'

x = []
y = []

count = 0
for (dirpath, dirnames, filenames) in os.walk(mypath):
  dirpath_list = dirpath.split('\\')
  # if count > 20:
  #   break


  if len(dirpath_list) > 5:
    for f in filenames:
      if ".jpg" in f:

        if 'pine-bottom' in dirpath:
        # if 'pine-side' in dirpath:
          count += 1
          # if count > 20:
          #   break

          print(f, dirnames, dirpath_list[3],{dirpath_list[4]}, dirpath_list[5])
          image_path = f'{dirpath}/{f}'
          image = keras.utils.load_img(image_path, color_mode="grayscale", target_size=(360, 640))
          input_arr = np.array([keras.utils.img_to_array(image)])  # Convert single image to a batch.
          x.append(input_arr)
          y.append(f"{dirpath_list[3]}_{dirpath_list[4]}_{dirpath_list[5]}_{f}")


01.jpg [] 0015 {'loc-2'} pine-bottom
03.jpg [] 0017 {'loc-2'} pine-bottom
02.jpg [] 0050 {'loc-1'} pine-bottom
01.jpg [] 0063 {'loc-2'} pine-bottom
03.jpg [] 0067 {'loc-1'} pine-bottom
02.jpg [] 0071 {'loc-2'} pine-bottom
04.jpg [] 0071 {'loc-2'} pine-bottom
01.jpg [] 0077 {'loc-1'} pine-bottom
02.jpg [] 0077 {'loc-1'} pine-bottom
03.jpg [] 0077 {'loc-1'} pine-bottom
01.jpg [] 0088 {'loc-2'} pine-bottom
01.jpg [] 0089 {'loc-2'} pine-bottom
03.jpg [] 0089 {'loc-2'} pine-bottom
01.jpg [] 0096 {'loc-1'} pine-bottom
04.jpg [] 0099 {'loc-1'} pine-bottom
04.jpg [] 0106 {'loc-2'} pine-bottom
01.jpg [] 0107 {'loc-2'} pine-bottom
03.jpg [] 0107 {'loc-2'} pine-bottom
01.jpg [] 0109 {'loc-2'} pine-bottom
03.jpg [] 0115 {'loc-1'} pine-bottom
01.jpg [] 0119 {'loc-2'} pine-bottom
03.jpg [] 0130 {'loc-2'} pine-bottom
01.jpg [] 0139 {'loc-1'} pine-bottom
02.jpg [] 0139 {'loc-1'} pine-bottom
01.jpg [] 0145 {'loc-1'} pine-bottom
01.jpg [] 0150 {'loc-2'} pine-bottom
02.jpg [] 0150 {'loc-2'} pine-bottom
0

### 訓練資料整合

#### train_label

In [3]:

train_label = pd.read_csv(f'{mypath}/training_set_label.csv')
print(train_label)
train_label.rename(columns={'1': 'ID', '1.1': 'Classification'}, inplace=True)

# 要插入的行的數據
new_row_data = {'ID': 1, 'Classification': 1}
new_row_df = pd.DataFrame([new_row_data])

# concat 函數將新的行插入到 DataFrame 的最開頭
train_label = pd.concat([new_row_df, train_label], ignore_index=True)

train_label.to_csv('train_label.csv', index=False)
train_label

       1  1.1
0      2    1
1      3    1
2      4    1
3      6    1
4      7    1
..   ...  ...
394  495    4
395  496    3
396  497    4
397  498    4
398  500    4

[399 rows x 2 columns]


,ID,Classification
0,1,1
1,2,1
2,3,1
3,4,1
4,6,1
...,...,...
395,495,4
396,496,3
397,497,4
398,498,4


#### train_y

In [4]:
# 轉換 y 為 DataFrame
y_df = pd.DataFrame({'Folder': y})
y_df
# 函數，提取 Folder 的前四個字符
def extract_prefix(folder):
    return int(folder[:4])

# ID = Folder 的前四個字符
y_df['ID'] = y_df['Folder'].apply(extract_prefix)
y_df.to_csv('y_img_test_data.csv', index=False)

In [5]:

# sort y_df['ID']
y_df = y_df.sort_values(by='ID')
y_df = y_df.reset_index()

# 將 train_label 中的 'Classification' 列值賦值給 y_df 中的 'Classification' 列
Classification = []
i = 0
j = 0

numi = len(y_df)
numj = len(train_label)

while (i < numi)&(j < numj):
  if y_df['ID'][i] == train_label['ID'][j]:
    Classification.append(train_label['Classification'][j])
    i = i+1
  elif y_df['ID'][i] < train_label['ID'][j]:
    i = i+1
    Classification.append('NaN')
  else:
    j = j+1

# 顯示修改後的 DataFrame
y_df['Classification'] = Classification

# 將 y_df 回復順序與 x_df 一致
y_df = y_df.sort_values(by='index')
y_df = y_df.reset_index(drop=True)

# 刪除 index 行
y_df = y_df.drop(columns=['index'])

# 新增 x_df['Classification']
Classification = y_df['Classification']
print(f'Classification.shape: ', Classification.shape)

# 刪除 y_df['Classification'] = 'NaN'
y_df = y_df.dropna(subset=['Classification'])
print('刪除 y_df[Classification] = NaN')
print(y_df.shape)

# 分類成 6 組
y_df12 = y_df[(y_df.Classification == 1) | (y_df.Classification == 2)]
y_df13 = y_df[(y_df.Classification == 1) | (y_df.Classification == 3)]
y_df14 = y_df[(y_df.Classification == 1) | (y_df.Classification == 4)]

y_df23 = y_df[(y_df.Classification == 2) | (y_df.Classification == 3)]
y_df24 = y_df[(y_df.Classification == 2) | (y_df.Classification == 4)]
y_df34 = y_df[(y_df.Classification == 3) | (y_df.Classification == 4)]


# y_data.csv
y_df.to_csv('y_data.csv', index=False)

y_df12.to_csv('y_data12.csv', index=False)
y_df13.to_csv('y_data13.csv', index=False)
y_df14.to_csv('y_data14.csv', index=False)

y_df23.to_csv('y_data23.csv', index=False)
y_df24.to_csv('y_data24.csv', index=False)
y_df34.to_csv('y_data34.csv', index=False)
print('y data complete!')


Classification.shape:  (101,)
刪除 y_df[Classification] = NaN
(101, 3)
y data complete!


#### train_x

x_padded

In [6]:
# 記錄每個數組的形狀
# shapes = [array.shape for array in x]
# print("x shape:", shapes)

# 將數據和標籤轉換為 NumPy 數組
# 找到最大形狀
max_shape = np.max([array.shape for array in x], axis=0)

# 使用 np.pad 將每個數組填充到相同的形狀
x_padded = [np.pad(array, [(0, max_shape[0] - array.shape[0]), (0, max_shape[1] - array.shape[1]), (0, max_shape[2] - array.shape[2]), (0, max_shape[3] - array.shape[3])], mode='constant') for array in x]

# 將填充後的列表轉換為 NumPy 數組
x_padded = np.array(x_padded)

# 打印填充後的 x 的形狀
print("Padded x shape:", x_padded.shape)

Padded x shape: (101, 1, 360, 640, 1)


轉為 DataFrame 並將分類結果=null的去掉

In [7]:
# 將四維陣列轉換為二維 DataFrame
x_reshaped = x_padded.reshape((x_padded.shape[0], -1))
x_df = pd.DataFrame(x_reshaped)

print("DataFrame 的形狀:", x_df.shape)
x_df.to_csv('x_img_test_data.csv', index=False)

DataFrame 的形狀: (101, 230400)


In [8]:

# 刪除 x_df['Classification'] = 'NaN' 的列
x_df['Classification'] = Classification
print('x_df shape: ', x_df.shape)

x_df = x_df.dropna(subset=['Classification'])
print('刪除 x_df[Classification] = NaN 的列後 x_df shape: ', x_df.shape)

# 分類成6組
x_df12 = x_df[(x_df.Classification == 1) | (x_df.Classification == 2)]
x_df13 = x_df[(x_df.Classification == 1) | (x_df.Classification == 3)]
x_df14 = x_df[(x_df.Classification == 1) | (x_df.Classification == 4)]

x_df23 = x_df[(x_df.Classification == 2) | (x_df.Classification == 3)]
x_df24 = x_df[(x_df.Classification == 2) | (x_df.Classification == 4)]
x_df34 = x_df[(x_df.Classification == 3) | (x_df.Classification == 4)]

# x_data.csv
x_df.to_csv('x_data.csv', index=False)

x_df12.to_csv('x_data12.csv', index=False)
x_df13.to_csv('x_data13.csv', index=False)
x_df14.to_csv('x_data14.csv', index=False)

x_df23.to_csv('x_data23.csv', index=False)
x_df24.to_csv('x_data24.csv', index=False)
x_df34.to_csv('x_data34.csv', index=False)
print('x data complete!')

x_df shape:  (101, 230401)
刪除 x_df[Classification] = NaN 的列後 x_df shape:  (101, 230401)
x data complete!
